<a href="https://colab.research.google.com/github/raminnourizade/Proposal/blob/main/egg_ds004504.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mne

In [2]:
 !pip install -q kaggle

In [3]:
from google.colab import files
import pandas as pd
import numpy as np
import os
import mne
import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras import layers, models
from keras.utils import to_categorical
import matplotlib.pyplot as plt


In [4]:
files.upload()


KeyboardInterrupt: 

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d bahoho/alzheimer-derivative-ds004504

In [ ]:
! mkdir train

In [ ]:
! unzip '/content/alzheimer-derivative-ds004504.zip' -d train



In [5]:
def get_set_files(root_dir):
    pattern = os.path.join(root_dir, '**', '*.set')
    set_files = glob.glob(pattern, recursive=True)
    return set_files


In [6]:
directory_path = '/content/train/derivatives'
set_files = get_set_files(directory_path)
# files_and_directories =  os.listdir(directory_path)
# directories=[d for d in files_and_directories if os.path.isdir(os.path.join(directory_path, d)) and d.startswith('sub')]


In [7]:
set_files

['/content/train/derivatives/sub-008/eeg/sub-008_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-032/eeg/sub-032_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-088/eeg/sub-088_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-046/eeg/sub-046_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-026/eeg/sub-026_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-010/eeg/sub-010_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-062/eeg/sub-062_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-074/eeg/sub-074_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-016/eeg/sub-016_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-048/eeg/sub-048_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-021/eeg/sub-021_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-043/eeg/sub-043_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-024/eeg/sub-024_task-eyesclosed_eeg.set',
 '/content/train/derivati

In [8]:
preprocessed_data = []

desired_length = 300*500
min_duration_seconds=100000

for file_path in set_files:
     # Load EEG data
    raw = mne.io.read_raw_eeglab(file_path)

    # Preprocess EEG data
    sampling_rate = raw.info['sfreq']
    print("Sampling Rate:", sampling_rate, "Hz")

    # Get duration of the recording
    duration_seconds = raw.times[-1]  # Duration is the last time point
    print("Duration:", duration_seconds, "seconds")
    if duration_seconds<min_duration_seconds: min_duration_seconds= duration_seconds
    # Example preprocessing steps:

    # 1. Filtering (e.g., bandpass filter)
    raw.filter(l_freq=0.5, h_freq=40)

    # 2. Resampling
    raw.resample(100)  # Resample to 100 Hz

    # 3. Extract data
    data = raw.get_data()

    # Pad or trim data to desired length
    if data.shape[1] < desired_length:
        # Pad data if shorter than desired length
        padded_data = np.pad(data, ((0, 0), (0, desired_length - data.shape[1])), mode='constant')
        preprocessed_data.append(padded_data)
    elif data.shape[1] > desired_length:
        # Trim data if longer than desired length
        trimmed_data = data[:, :desired_length]
        preprocessed_data.append(trimmed_data)
    else:
        preprocessed_data.append(data)

# Convert preprocessed_data to numpy array
preprocessed_data = np.array(preprocessed_data)

<ipython-input-8-6aa1169c2bfa>:8: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = mne.io.read_raw_eeglab(file_path)


Sampling Rate: 500.0 Hz
Duration: 793.398 seconds
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (6.602 s)



<ipython-input-8-6aa1169c2bfa>:8: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(file_path)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.4s


Sampling Rate: 500.0 Hz
Duration: 797.778 seconds
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (6.602 s)



<ipython-input-8-6aa1169c2bfa>:8: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = mne.io.read_raw_eeglab(file_path)
<ipython-input-8-6aa1169c2bfa>:8: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(file_path)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


...

<ipython-input-8-6aa1169c2bfa>:8: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = mne.io.read_raw_eeglab(file_path)
<ipython-input-8-6aa1169c2bfa>:8: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(file_path)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.4s


Sampling Rate: 500.0 Hz
Duration: 618.898 seconds
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (6.602 s)



<ipython-input-8-6aa1169c2bfa>:8: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = mne.io.read_raw_eeglab(file_path)
<ipython-input-8-6aa1169c2bfa>:8: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(file_path)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


Sampling Rate: 500.0 Hz
Duration: 826.198 seconds
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (6.602 s)



<ipython-input-8-6aa1169c2bfa>:8: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = mne.io.read_raw_eeglab(file_path)
<ipython-input-8-6aa1169c2bfa>:8: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(file_path)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


In [9]:
dataset=pd.read_csv('/content/participants.tsv',delimiter='	')

In [10]:
labels_dict = dict(zip(dataset["participant_id"], dataset["Group"]))

In [11]:
labels_dict

{'sub-001': 'A',
 'sub-002': 'A',
 'sub-003': 'A',
 'sub-004': 'A',
 'sub-005': 'A',
 'sub-006': 'A',
 'sub-007': 'A',
 'sub-008': 'A',
 'sub-009': 'A',
 'sub-010': 'A',
 'sub-011': 'A',
 'sub-012': 'A',
 'sub-013': 'A',
 'sub-014': 'A',
 'sub-015': 'A',
 'sub-016': 'A',
 'sub-017': 'A',
 'sub-018': 'A',
 'sub-019': 'A',
 'sub-020': 'A',
 'sub-021': 'A',
 'sub-022': 'A',
 'sub-023': 'A',
 'sub-024': 'A',
 'sub-025': 'A',
 'sub-026': 'A',
 'sub-027': 'A',
 'sub-028': 'A',
 'sub-029': 'A',
 'sub-030': 'A',
 'sub-031': 'A',
 'sub-032': 'A',
 'sub-033': 'A',
 'sub-034': 'A',
 'sub-035': 'A',
 'sub-036': 'A',
 'sub-037': 'C',
 'sub-038': 'C',
 'sub-039': 'C',
 'sub-040': 'C',
 'sub-041': 'C',
 'sub-042': 'C',
 'sub-043': 'C',
 'sub-044': 'C',
 'sub-045': 'C',
 'sub-046': 'C',
 'sub-047': 'C',
 'sub-048': 'C',
 'sub-049': 'C',
 'sub-050': 'C',
 'sub-051': 'C',
 'sub-052': 'C',
 'sub-053': 'C',
 'sub-054': 'C',
 'sub-055': 'C',
 'sub-056': 'C',
 'sub-057': 'C',
 'sub-058': 'C',
 'sub-059': 'C

In [12]:
labels = [labels_dict[file_path.split('/')[-3]] for file_path in set_files]


In [13]:
le = LabelEncoder()
y = le.fit_transform(labels)

In [ ]:
y

In [14]:
X = preprocessed_data


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [22]:
def VisionTransformer(input_shape, num_classes):
    # ViT architecture
    inputs = layers.Input(shape=input_shape)

    # Embedding layer
    embedding = layers.Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(inputs)  # Adjust filters and kernel_size as needed
    embedding = layers.Reshape((-1, 64))(embedding)  # Flatten spatial dimensions

    # Transformer encoder
    num_layers = 8  # Increase the number of transformer encoder layers
    num_heads = 8   # Number of attention heads
    hidden_dim = 1024  # Increase hidden dimension of feedforward network
    dropout_rate = 0.2  # Adjust dropout rate

    for _ in range(num_layers):
        # Multi-head self-attention
        attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=hidden_dim // num_heads)(embedding, embedding)
        attention_output = layers.Dropout(dropout_rate)(attention_output)
        attention_output = layers.LayerNormalization()(embedding + attention_output)

        # Feedforward network
        feedforward_output = layers.Dense(hidden_dim, activation='relu')(attention_output)
        feedforward_output = layers.Dropout(dropout_rate)(feedforward_output)
        feedforward_output = layers.Dense(embedding.shape[-1])(feedforward_output)
        feedforward_output = layers.Dropout(dropout_rate)(feedforward_output)
        embedding = layers.LayerNormalization()(attention_output + feedforward_output)

    # Classification head
    output = layers.GlobalAveragePooling1D()(embedding)
    output = layers.Dense(num_classes, activation='softmax')(output)

    # Create model
    model = models.Model(inputs, output)
    return model

In [17]:
input_shape = X_train.shape[1:]  # Shape of input data (excluding batch dimension)
num_classes = len(np.unique(y_train))  # Number of classes

In [18]:
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [23]:
# Initialize ViT model
vit_model = VisionTransformer(input_shape, num_classes)

In [24]:
vit_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [25]:
vit_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/20
3/3 [==============================] - 61s 6s/step - loss: 3.6111 - accuracy: 0.3714 - val_loss: 1.7465 - val_accuracy: 0.5556
Epoch 2/20
3/3 [==============================] - 16s 4s/step - loss: 1.8113 - accuracy: 0.3000 - val_loss: 1.5310 - val_accuracy: 0.2222
Epoch 3/20
3/3 [==============================] - 16s 5s/step - loss: 1.3280 - accuracy: 0.2714 - val_loss: 1.1520 - val_accuracy: 0.2222
Epoch 4/20
3/3 [==============================] - 17s 6s/step - loss: 1.2382 - accuracy: 0.3143 - val_loss: 0.9964 - val_accuracy: 0.5556
Epoch 5/20
3/3 [==============================] - 13s 4s/step - loss: 1.1118 - accuracy: 0.3714 - val_loss: 1.1446 - val_accuracy: 0.2222
Epoch 6/20
3/3 [==============================] - 16s 5s/step - loss: 1.1234 - accuracy: 0.3571 - val_loss: 1.5131 - val_accuracy: 0.2222
Epoch 7/20
3/3 [==============================] - 14s 4s/step - loss: 1.2857 - accuracy: 0.3571 - val_loss: 1.4338 - val_accuracy: 0.2222
Epoch 8/20
3/3 [==================

In [26]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

NameError: name 'history' is not defined